##### Probabilistic Models – Spring 2022
## Exercise Session 3
Return by Feb 15th 12.00. Session at Feb 15th 14.15.

<span style="color:red">**Enrico Buratto**</span>

### Instructions
Make sure the notebook produces correct results when ran sequentially starting from the first cell. You can ensure this by clearing all outputs, running all cells, and finally correcting any errors.

To get points:
1. Submit your answers to the automatically checked Moodle test. 
 - You have 5 tries on the test: the highest obtained score will be taken into account.
 - For numerical questions the tolerance is $1\cdot10^{-4}$, so you can round the results e.g. up to 5 decimals.
2. Submit this notebook containing your derivations to Moodle.

The idea in this exercise is to understand how a naive Bayes classifier and a hidden Markov model work, by implementing them yourself. Therefore, you should not use any libraries providing the models off-the-shelf. Also you should not copy-paste any solutions you might find on the Internet. Doing so you will risk losing the points for the exercise. You can, of course, compare the output of your implementation to any reference implementation.

## Exercise 1
***

Consider the following document corpus, where the first column specifies the class of the document and the second column is the document itself.


In [1]:
!cat data/corpus.txt

Class	Text
spam	FREE online !!! results
ham	results repository online FREE
spam	FREE online results FREE !!! registration
spam	!!! registration FREE !!! repository
ham	conference online registration conference !!!
ham	conference results repository results FREE


(a) Construct a multinomial naive Bayes classifier (NBC; p. 25, lecture 5) using the corpus as the training data. Use Laplace smoothing to address the problem of zero probabilities. 

Report in Moodle:
- $P(C=\text{ham})$,
- $P(X=\text{repository} \mid C=\text{spam})$.



(b) Use the NBC to calculate the posterior probability for each document that it is spam. 

Report in Moodle:
- the obtained value for the last document in the corpus.

(c) Finally, compute and report in Moodle the posterior probabilities for the following documents (not used in the training of the NBC) to be spam:

In [2]:
!cat data/new_emails.txt

Class	Text
?	FREE online conference !!! registration
?	conference registration results conference online !!!


For each question asking for a posterior probability, remember to normalize so that your posterior probabilities sum up to 1.

In [48]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore') # probably using something deprecated but works

# take distinct words
corpus = pd.read_csv('data/corpus.txt', sep='\t')
words = set()
corpus['Text'].str.split().apply(words.update)
words = list(words)

# count occurrences
df = pd.DataFrame(columns=['word','spam','ham'])
for word in words:
    count_spam = corpus[corpus['Class']=='spam']['Text'].str.count(word).sum()
    count_ham = corpus[corpus['Class']=='ham']['Text'].str.count(word).sum()
    df = df.append({'word': word, 'spam': count_spam, 'ham': count_ham}, ignore_index=True)

# count probabilities (+1 for laplacian smoothing)
df['spam'] = (df['spam'] + 1)/(df['spam'].sum() + len(words))
df['ham'] = (df['ham'] + 1)/(df['ham'].sum() + len(words))

# Task a
print('Task a')
print('P(C=ham) = 0.5') # this is just number of ham/total
print('P(X=repository|C=spam) =', df[df['word']=='repository']['spam'].values[0])

# Task b

def calculatePosterior(data, df):
    last = data
    spam_given_d = 1
    for word in last.split():
        spam_given_d *= df[df['word']==word]['spam'].values[0]
    spam_given_d *= .5
    ham_given_d = 1
    for word in last.split():
        ham_given_d *= df[df['word']==word]['ham'].values[0]
    ham_given_d *= .5
    return spam_given_d/(spam_given_d+ham_given_d)

print('Task b')
print('P(C=spam|D) =', calculatePosterior(corpus['Text'].iloc[-1], df))

# Task c
test = pd.read_csv('data/new_emails.txt', sep='\t')

# first one
print('Task c')
post = calculatePosterior(test['Text'].iloc[0], df)
print('P(C=spam|first) =', post)
print('P(C=ham|first) =', 1-post)

post = calculatePosterior(test['Text'].iloc[1], df)
print('P(C=spam|second) =', post)
print('P(C=ham|second) =', 1-post)

Task a
P(C=ham) = 0.5
P(X=repository|C=spam) = 0.09090909090909091
Task b
P(C=spam|D) = 0.11018056221333225
Task c
P(C=spam|first) = 0.5532194007327792
P(C=ham|first) = 0.4467805992672208
P(C=spam|second) = 0.1173637534202318
P(C=ham|second) = 0.8826362465797682


## Exercise 2
***

**NB**: The exercise does not assume knowledge on molecular biology. The description shows one simplified use case for a hidden Markov model.

The central dogma of molecular biology (roughly) states that DNA is transcribed into RNA. The RNA is then translated into protein, and proteins perform most of the functions within the cell. A protein is composed of a string of amino acids. While there are about twenty commonly occuring amino acids, and they can be grouped many different ways, we will consider them all to belong to either the Aliphatic or Hydroxyl group (to simplify calculations). This sequence is called the primary structure of the protein. A key factor in understanding the function of a particular protein is its secondary structure, that is, how the amino acids behave in three dimensional space. For example, the following figure shows a cartoon rendering of the secondary structure of a protein.

![](secondarystructure.jpg)

As the figure suggests, two of the most important features of the seconary structure are $\alpha$ helices and $\beta$ sheets. However, it is difficult to assess the secondary structure without expensive experimental techniques such as crystallography. On the other hand, finding the sequence of amino acids in a protein sequence is inexpensive. Consequently, a goal of computational biology is to infer the secondary structure of a protein given its primary structure (the sequence of amino acids).

Suppose we conduct a set of costly crystallography experiments and discover the following secondary structures and associated amino acid sequences. $\alpha$ means $\alpha~\text{helix}$, $\beta$ means $\beta~\text{sheet}$, $\mathtt{A}$ means aliphatic and $\mathtt{H}$ means hydroxyl. For example, in the first protein (i.e., first row) the second $\beta$ corresponds to the third $\mathtt{A}$, because they are both in the third position.

In [4]:
!cat data/sequences.txt

αββααββ AAAHHHA
αααβααβ HHAAHHH
βαααββ AAAAHA
αββαααβα AHAAAHAA


(a) Use these sequences to calculate the parameters for a hidden Markov model (HMM) in which the state variables correspond to the secondary structures and the observations correspond to the amino acids. We will use $S_i$ and $O_i$ to denote the secondary structure and the amino acid in position $i$, respectively. Recall that for an HMM, we need:

- The prior probabilities for the first state, $S_1$.  In this case, use the observed counts of the first state in each sequence.
- The *emission* probabilities, that is, the probability of observing a particular amino acid given the secondary structure.
- The *transition* probabilities, that is, the probability of the next state given the current state.

In all cases, use Laplace smoothing, as was done in Exercise 1.

Report in Moodle:

- the prior probability $P(S_1 = \alpha)$,
- the emission probabilities $P(O_i = \mathtt{A} \mid S_i = \alpha)$ and $P(O_i = \mathtt{H} \mid S_i = \beta)$,
- the transition probabilities $P(S_{i+1} = \beta \mid S_i = \alpha)$ and $P(S_{i+1} = \alpha \mid S_i = \beta)$.

(b) Now, using the HMM, use the foward-backward algorithm to calculate the posterior probabilities over all the secondary structure states, given the following observed amino acid sequence:

- $\mathtt{AHHAAHAAHA}$

Report in Moodle:
- $P(S_6 = \alpha \mid O_{1:10})$, i.e., the posterior probability for the 6th secondary state (corresponding to the third $\mathtt{H}$ in the sequence) to be $\alpha$.

(c) Finally, implement and use the Viterbi algorithm to calculate the most likely sequence of secondary structure states for the sequence of observations used in (b).

Report in Moodle:
- The obtained sequence. If there are multiple sequences with equally high likelihood, report any one of them. To avoid possible problems with character encoding use a and b to denote $\alpha$ and $\beta$: if you obtain the sequence $\alpha\alpha\alpha\alpha\alpha\beta\beta\beta\beta\beta$ report it as aaaaabbbbb.

Again, remember to normalize so that the posterior probabilities sum up to 1.

In [5]:
# Provide your answer in cells here